L'objetif de ce tp va être de découvrir les fonctionnalités d'Elasticsearch et de Kibana afin de construire un premier tableau de bord qui va permettre d'explorer un nouveau jeu de données.

In [ ]:
# !conda install Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch 
from elasticsearch import helpers
import pandas as pd

Affichez les infos sur le cluster

In [ ]:
# création client pour se connecter au serveur Elasticsearch (qui tourne sur le Docker) 
es_client=Elasticsearch([{'host':'localhost','port':9200}])
es_client.cluster.health()

# Init and Create

Téléchargez ces données et importez les dans un dataframe avec pandas
https://www.kaggle.com/roshansharma/sanfranciso-crime-dataset?select=Police_Department_Incidents_-_Previous_Year__2016_.csv

In [ ]:
# TODO lire le csv à l'aide de pandas
df = ...

In [ ]:
df.shape

In [ ]:
df.head(2)

### Création de l'index vide

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

# TODO créer un index (vide pour l'instant) "sf_crimes" dans lequel on rajoutera les données
# utiliser indices.create

In [ ]:
# TODO Visualisation des settings de l'index


In [ ]:
# TODO On affiche le mapping de l'index


# Premier Insert

A l'aide de l'api helpers.bulk insérez tout le dataframe dans l'index que l'on vient de créer.

In [ ]:
es_client = Elasticsearch(http_compress=True)


def filterKeys(document, columns):
    return {key: document[key] for key in columns }

# TODO : remplacez les "?"
# on utilise ici un générateur pour rajouter les documents à l'index
def doc_generator(df, INDEX):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": ??,
                "_type": "_doc",
                "_id" : f"{??}",
                "_source": filterKeys(??, ??),
            }
    raise StopIteration


In [ ]:
# utilisation de l'api bulk pour rakouter plusieurs documents au même temps
helpers.bulk(es_client, doc_generator(df ,INDEX))

In [ ]:
# Ca plante car nous n'avions pas géré les nan

In [ ]:
df.isna().sum()

In [ ]:
df_cleaned = df.where(df.notna(), lambda x: [None]) #Ici on permet à json de comprendre que c'est un Nan avec None

In [ ]:
# Comme on a inséré à partir des indexs, nous ne sommes pas obligés de tout supprimer. Le Bulk va réécrire et donc ne pas modifier ce qui existe déjà en base

In [ ]:
helpers.bulk(es_client, doc_generator(df_cleaned ,INDEX))

In [ ]:
# On vérifie l'insert
es.indices.refresh(INDEX)
es.cat.count(INDEX, params={"format": "json"})

In [ ]:
es.indices.refresh(INDEX)

#TODO On affiche le mapping dynamique utilisé par défault

On peut s'apercevoir que le mapping par défaut n'est pas judicieux. Effectivement la date n'est pas traîtée comme telle. Il existe pour X et Y un type geopoint. 
Il va donc nous falloir créer un mapping et réinsérer

# On ajoute un mapping

## Date
Ajoutez un mapping pour prendre en compte les dates

In [ ]:
es_client.indices.delete(INDEX)

In [ ]:
df.head(1)

pour les dates se baser sur : https://www.joda.org/joda-time/apidocs/org/joda/time/format/DateTimeFormat.html

In [ ]:
# TODO Créez un dict python qui contient le mapping que vous affectez dans la cellule suivante
# format date : M/d/yyyy K:m:s a
# type pour coord gps : geo_point


In [ ]:
es_client.indices.create(index=INDEX, body=mapping)

In [ ]:
# TODO: Affichez le mapping et vérifiez 

In [ ]:
#Test
helpers.bulk(es_client, doc_generator(_df_cleaned.head(1) ,INDEX))

# Geopoint
https://www.elastic.co/guide/en/elasticsearch/reference/current/geo-point.html

In [ ]:
# TODO : Faites de même que ce tuto pour pouvoir traîter Localisation comme un Geopoint

# Kibana 

### Ajout de l'index

Maintenant que les données sont bien formées, vous devez ajouter l'index sf_crimes à Kibana.
Pour ce faire allez sur http://localhost:5601/ et cliquez à gauche sur "Stack Management" puis Index pattern et ajoutez sf_crimes*.
Vérifiez dans la fenêtre suivante que le mapping est bon.

### Création d'une première visualisation 

Maintenant vous allez pouvoir commencer à créer plusieurs visualisations que vous rassemblerez dans un dashboard. On va commencer ma créer une carte qui géolocalise les crimes.
Pour cela cliquez sur Visualize puis new. Je vous laisse chercher ;)

# Création d'un dashboard

Une fois que vous avez plusieurs visualisations, vous pouvez vous créer un dashboard en ajoutant dans un nouveau dashboard les visualisations que vous avez préalablement sauvegardé.

Bonne exploration :)